## Deliverable 2. Create a Customer Travel Destinations Map.

In [ ]:
pip install gmaps

In [ ]:
pip install ipywidgets

In [1]:
# Dependencies and Setup
import matplotlib
%matplotlib inline
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.6001,-73.9662,55.00,84,99,2.35,overcast clouds
1,1,Tiksi,RU,71.6872,128.8694,19.78,84,83,8.48,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,55.29,96,55,7.67,broken clouds
3,3,Kodiak,US,57.7900,-152.4072,37.40,87,90,11.50,moderate rain
4,4,Hobart,AU,-42.8794,147.3294,64.40,42,75,28.77,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 40


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kodiak,US,57.7900,-152.4072,37.40,87,90,11.50,moderate rain
13,13,Neryungri,RU,56.6709,124.6491,22.37,69,79,3.15,broken clouds
43,43,Haines Junction,CA,60.7522,-137.5108,37.40,60,98,6.91,overcast clouds
46,46,Skibbereen,IE,51.5500,-9.2667,39.20,100,75,3.44,fog
48,48,Okhotsk,RU,59.3833,143.3000,24.96,55,100,11.25,overcast clouds
63,63,Ilulissat,GL,69.2167,-51.1000,21.20,93,90,11.50,overcast clouds
68,68,Nikolskoye,RU,59.7035,30.7861,39.99,75,90,8.95,light rain
73,73,Nivala,FI,63.9167,24.9667,32.23,88,79,7.94,broken clouds
78,78,Tasiilaq,GL,65.6145,-37.6368,28.40,74,49,9.22,scattered clouds
82,82,Bronnoysund,NO,65.4625,12.1997,37.40,87,75,12.66,broken clouds


In [5]:

preferred_cities_df = preferred_cities_df.dropna()

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                124
City                   124
Country                124
Lat                    124
Lng                    124
Max Temp               124
Humidity               124
Cloudiness             124
Wind Speed             124
Current Description    124
dtype: int64

In [7]:

# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#preferred_cities_df.dropna() 

In [8]:

# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kodiak,US,37.40,moderate rain,57.7900,-152.4072,
13,Neryungri,RU,22.37,broken clouds,56.6709,124.6491,
43,Haines Junction,CA,37.40,overcast clouds,60.7522,-137.5108,
46,Skibbereen,IE,39.20,fog,51.5500,-9.2667,
48,Okhotsk,RU,24.96,overcast clouds,59.3833,143.3000,
...,...,...,...,...,...,...,...
663,Kirensk,RU,35.20,light snow,57.7853,108.1119,
691,Anaconda,US,24.80,overcast clouds,46.1285,-112.9423,
698,Sosnovo-Ozerskoye,RU,36.57,overcast clouds,52.5249,111.5418,
702,Sorland,NO,33.80,broken clouds,67.6670,12.6934,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}" 
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    # 6e. Make request and retrieve the JSON data from the search. 
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel Found")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel not found... skipping.
Hotel not found... skipping.
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel Found
Hotel n

In [10]:
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kodiak,US,37.40,moderate rain,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
13,Neryungri,RU,22.37,broken clouds,56.6709,124.6491,Pyat' Zvozd
43,Haines Junction,CA,37.40,overcast clouds,60.7522,-137.5108,Parkside Inn
46,Skibbereen,IE,39.20,fog,51.5500,-9.2667,The West Cork Hotel
63,Ilulissat,GL,21.20,overcast clouds,69.2167,-51.1000,Hotel Arctic
...,...,...,...,...,...,...,...
654,Asvanyraro,HU,39.00,overcast clouds,47.8273,17.4942,Orchidea Hotel Lipót
663,Kirensk,RU,35.20,light snow,57.7853,108.1119,Kirenskiy Plos
691,Anaconda,US,24.80,overcast clouds,46.1285,-112.9423,Trade Wind Motel
702,Sorland,NO,33.80,broken clouds,67.6670,12.6934,Sjybrygga fiskecamp


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df;

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations= clean_hotel_df[["Lat", "Lng"]]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kodiak,US,37.40,moderate rain,57.7900,-152.4072,Best Western Kodiak Inn And Convention Center
13,Neryungri,RU,22.37,broken clouds,56.6709,124.6491,Pyat' Zvozd
43,Haines Junction,CA,37.40,overcast clouds,60.7522,-137.5108,Parkside Inn
46,Skibbereen,IE,39.20,fog,51.5500,-9.2667,The West Cork Hotel
63,Ilulissat,GL,21.20,overcast clouds,69.2167,-51.1000,Hotel Arctic
68,Nikolskoye,RU,39.99,light rain,59.7035,30.7861,Tourist House - Sablino
73,Nivala,FI,32.23,broken clouds,63.9167,24.9667,Hotelli Huiskankorpi / Aseman majatalo
78,Tasiilaq,GL,28.40,scattered clouds,65.6145,-37.6368,The Red House
82,Bronnoysund,NO,37.40,broken clouds,65.4625,12.1997,Corner Hotel
85,Inuvik,CA,30.20,broken clouds,68.3499,-133.7218,Nova Inn Inuvik


In [14]:
gmaps.configure(api_key=g_key)

In [15]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Save figure
import matplotlib.pyplot as plt
plt.savefig("Vacation_Search/Hotel_listings.png")